Image stack alignment (moving -> fixed)
- For each moving plane, find best (x, y, z, scale) in fixed stack.
- Coarse-to-fine pyramid with template matching + subpixel refinement.
- Outputs per-plane mapping in pixels (x,y) in fixed coords, z index/µm, and scale.

Johannes Larsch 20250916

In [ ]:
# Setup & imports
%load_ext autoreload
%autoreload 2
import sys, math, warnings, json
import numpy as np

import os

# Get parent of the notebook dir (project_root)
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add sibling directory "src" to sys.path
sys.path.append(os.path.join(project_root, "src"))

import alignSubstackUtils as asu
import saveFunctionalProjections as sfp
from skimage.transform import rotate as sk_rotate

import pandas as pd


from pathlib import Path
import numpy as np
import tifffile as tiff


_HAS_SKIMAGE = True


In [ ]:
# === EDIT THIS PATH ===
START_FOLDER = Path(r"D:/i/danin_tests/temp/planes/")
rotation_angle = 140  # degrees, e.g. 0, 90, 180, 270
sfp.save_max_avg_projections(START_FOLDER, rotation_angle)


# Example usage:



In [ ]:
# %% [markdown]
# ### Example usage
# Replace the dummy arrays with your actual data (NumPy arrays loaded from TIFF, NIfTI, HDF5, etc.)

# %%


# # Run registration
# df_results = register_moving_stack(
#     fixed_stack,
#     moving_stack,
#     fixed_z_spacing_um=1.0,
#     scale_range=(0.35, 1.0),   # moving is more zoomed-in -> usually < 1.0
#     n_scales=11,
#     z_stride_coarse=4,
#     z_refine_radius=3,
#     pyramid_downscale=2,
#     pyramid_min_size=160,
#     verbose=True,
# )
# display(df_results)

# %% [markdown]
# ### Notes & tips
# - If the moving FOV is substantially smaller, keep `scale_range` < 1 (e.g., 0.2–0.8).
# - If computation is slow, increase `z_stride_coarse` or `pyramid_min_size`.
# - For very noisy data, consider pre-filtering with a small Gaussian before normalization.
# - `ncc_score` near 1.0 indicates a strong match; inspect low scores visually.
# - Coordinates (x_px, y_px) are given in **fixed** image pixels at full resolution.



In [ ]:
import tifffile

moving_p = Path(r"Y:/Danin/imaging/temp/planes/projections/avg_projections.tif")
fixed_p = Path(r"Y:/Danin/imaging/temp/f38_anatomy_00001_rotate_8b.tif")
fixed_stack = tifffile.imread(fixed_p)   # shape: (Zf, Yf, Xf), spacing 1 µm
moving_stack = tifffile.imread(moving_p) # shape: (Zm, Ym, Xm), spacing 4–15 µm between planes


df_results = asu.register_moving_stack(
    fixed_stack, 
    moving_stack,
    fixed_z_spacing_um=1.0,     # spacing between planes in the fixed stack (µm)
    scale_range=(0.6, 1.0),   # expected relative zoom (moving→fixed); narrower range boosts SNR
    n_scales=10,                 # number of discrete scales to test between scale_range
    pyramid_downscale=2,      # downscale factor per pyramid level (smaller = finer refinement)
    pyramid_min_size=140,       # stop pyramid when min dimension < this; ensures at least 1 coarse level
    z_stride_coarse=2,          # step size for z-search at coarsest level (higher = faster, lower = more exhaustive)
    z_refine_radius=3,          # number of planes around best z to test at finer levels
    verbose=True,               # print per-plane progress and NCC scores
)

display(df_results)

In [ ]:
asu.interactive_checker(fixed_stack, moving_stack, df_results)